![lenet结构图](../workspace/pic/lenet.jpg)
1. 卷积的输出形状由批量大小、通道数、高度、宽度决定
2. 每个2x2池化层将空间维数减少4倍
3. 3个全连接层，分别有120、84、10个输出

In [10]:
import torchvision
import torch
from torch.utils.data import DataLoader
from torch import nn
from torch.utils.tensorboard import SummaryWriter


class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        net = nn.Sequential(
        nn.Conv2d(1,6,kernel_size=5,padding=2),  #根据特征图计算公式
        nn.Sigmoid(),
        nn.AvgPool2d(kernel_size=2,stride=2), 
        nn.Conv2d(6, 16, kernel_size=5),
        nn.Sigmoid(),
        nn.AvgPool2d(kernel_size=2, stride=2),
        nn.Flatten(),
        nn.Linear(16*5*5,120),
        nn.Sigmoid(),
        nn.Linear(120,84),
        nn.Sigmoid(),
        nn.Linear(84,10)
        )
        
    def forward(self, x):
        x = self.model1(x)
        return x
    
if __name__ == 'main':
    net = LeNet()
    input = torch.ones((64, 3, 32, 32))
    output = net(input)
    print(output.shape)


In [11]:
LeNet()


LeNet()

In [12]:
train_data = torchvision.datasets.FashionMNIST(root = "./data", train=True, transform=torchvision.transforms.ToTensor(),
                                         download = True)
test_data = torchvision.datasets.FashionMNIST(root = "./data", train=True, transform=torchvision.transforms.ToTensor(),
                                         download = True)

ValueError: unknown url type: '/media/zqh/planB/Phoenix_Z/Learning/doc/DeepLearning/data/train-images-idx3-ubyte.gz'